# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
from pandas import DataFrame, Series
from sklearn import linear_model
from sklearn.metrics import mean_squared_error 

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float,
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str,
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float,
              'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int,
              'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
l1_penalty = 5e2
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = linear_model.Lasso(alpha = l1_penalty, normalize= True)
model_all.fit(sales[all_features], sales['price'])

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Find what features had non-zero weight.

In [8]:
def print_coef(model, predictors):
    '''
        custom function to print coefficients of LASSO model
    '''
    coef_vec = list(model.coef_)
    print("%-16s" % "Predictor", "| ", "Coefficient")
    print("-----------------------------------------")
    for i,p in enumerate(coef_vec):
        print("%-16s" % predictors[i], ": ", p)

print_coef(model_all, all_features)

Predictor        |  Coefficient
-----------------------------------------
bedrooms         :  0.0
bedrooms_square  :  0.0
bathrooms        :  0.0
sqft_living      :  134.439313955
sqft_living_sqrt :  0.0
sqft_lot         :  0.0
sqft_lot_sqrt    :  0.0
floors           :  0.0
floors_square    :  0.0
waterfront       :  0.0
view             :  24750.0045856
condition        :  0.0
grade            :  61749.1030907
sqft_above       :  0.0
sqft_basement    :  0.0
yr_built         :  -0.0
yr_renovated     :  0.0


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [9]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [11]:
l1_pen_vec = np.logspace(1, 7, num=13)
rss_vec = np.zeros(len(l1_pen_vec))

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [13]:
for i,p in enumerate(l1_pen_vec):
    model = linear_model.Lasso(alpha = p, normalize= True)
    model.fit(training[all_features], training['price'])
    errors = model.predict(validation[all_features]) - validation['price']
    rss_vec[i] = np.sum(errors ** 2)

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [17]:
print(l1_pen_vec[rss_vec == min(rss_vec)])

[ 10.]


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [19]:
l1_pen_to_use = l1_pen_vec[(rss_vec == min(rss_vec))]
model = linear_model.Lasso(alpha = l1_pen_to_use, normalize= True)
model.fit(training[all_features], training['price'])
errors = model.predict(testing[all_features]) - testing['price']
rss_test = np.sum(errors ** 2)

print(np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))

15


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [20]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [43]:
l1_pen_vec = np.logspace(1, 4, num=20)
nonzero_weights_cnt = np.zeros(len(l1_pen_vec))
print(l1_pen_vec)

[    10.             14.38449888     20.69138081     29.76351442
     42.81332399     61.58482111     88.58667904    127.42749857
    183.29807108    263.66508987    379.26901907    545.55947812
    784.75997035   1128.83789168   1623.77673919   2335.72146909
   3359.81828628   4832.93023857   6951.92796178  10000.        ]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [36]:
for i,p in enumerate(l1_pen_vec):
    model = linear_model.Lasso(alpha = p, normalize= True)
    model.fit(training[all_features], training['price'])
    nonzero_weights_cnt[i] = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

    # any smaller than pen_min will give us more non-zeros than required

print(nonzero_weights_cnt)

[ 15.  15.  15.  15.  13.  12.  11.  10.   7.   6.   6.   6.   5.   3.   3.
   2.   1.   1.   1.   1.]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [41]:
l1_pen_min = max(l1_pen_vec[nonzero_weights_cnt > max_nonzeros])
l1_pen_max = min(l1_pen_vec[nonzero_weights_cnt < max_nonzeros])

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

In [39]:
print(round(l1_pen_min,2))

127.43


In [40]:
print(round(l1_pen_max,2))

263.67


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [52]:
l1_penalty_values = np.linspace(l1_pen_min,l1_pen_max,20)
nonzero_weights_cnt_2 = np.zeros(len(l1_penalty_values))
rss_vec = np.zeros(len(l1_penalty_values))
print(l1_penalty_values)

[ 127.42749857  134.59789811  141.76829765  148.9386972   156.10909674
  163.27949628  170.44989582  177.62029537  184.79069491  191.96109445
  199.13149399  206.30189354  213.47229308  220.64269262  227.81309216
  234.9834917   242.15389125  249.32429079  256.49469033  263.66508987]


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [53]:
for i,p in enumerate(l1_penalty_values):
    model = linear_model.Lasso(alpha = p, normalize= True)
    model.fit(training[all_features], training['price'])
    nonzero_weights_cnt_2[i] = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    errors = model.predict(validation[all_features]) - validation['price']
    rss_vec[i] = np.sum(errors ** 2)

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [55]:
rss_choice = min(rss_vec[nonzero_weights_cnt_2 == max_nonzeros])
l1_pen_choice = l1_penalty_values[rss_vec == rss_choice]
print(l1_pen_choice)

[ 156.10909674]


In [57]:
model = linear_model.Lasso(alpha = l1_pen_choice, normalize = True)
model.fit(training[all_features], training['price'])
print_coef(model, all_features)

Predictor        |  Coefficient
-----------------------------------------
bedrooms         :  -0.0
bedrooms_square  :  -0.0
bathrooms        :  10610.8902844
sqft_living      :  163.380251648
sqft_living_sqrt :  0.0
sqft_lot         :  -0.0
sqft_lot_sqrt    :  -0.0
floors           :  0.0
floors_square    :  0.0
waterfront       :  506451.687115
view             :  41960.0435549
condition        :  0.0
grade            :  116253.5537
sqft_above       :  0.0
sqft_basement    :  0.0
yr_built         :  -2612.23488036
yr_renovated     :  0.0
